# Exercise 1

## **Table**:
- #### `dbo.Orders`

### **Description**:<br>A query that computes both a rank and a dense rank for each customer order, partitioned by `custid` and `qty`:

In [6]:
USE	[TSQLV6]
GO

SELECT
	custid	AS	[Customer ID],
	orderid	AS	[Order ID],
	qty		AS	[Quantity],
	RANK()
		OVER(PARTITION BY	custid	ORDER BY	qty)
			AS	[Rank],
	DENSE_RANK()
		OVER(PARTITION BY	custid	ORDER BY	qty)
			AS	[Dense Rank]
FROM
	dbo.Orders;

Commands completed successfully.

(11 rows affected)

Total execution time: 00:00:00.012

Customer ID,Order ID,Quantity,Rank,Dense Rank
A,30001,10,1,1
A,40005,10,1,1
A,10001,12,3,2
A,40001,40,4,3
B,20001,12,1,1
B,30003,15,2,2
B,10005,20,3,3
C,10006,14,1,1
C,20002,20,2,2
C,30004,22,3,3


# Exercise 2

## **Table**:
- #### `Sales.OrderValues`

### **Description**:<br>An alternative form of the following query:
```SQL
SELECT
	val     AS	[Value],
	ROW_NUMBER()
		OVER(ORDER BY	val)
			AS  [Row Number]
FROM
	Sales.OrderValues
GROUP BY
	val;
```

In [ ]:
USE	[TSQLV6]
GO

SELECT
	val		AS	[Value],
	ROW_NUMBER()
		OVER(ORDER  BY  val)
			AS	[Row Number]
FROM(
	SELECT  DISTINCT
		val
	FROM
		Sales.OrderValues
)			AS	[ov];


# Exercise 3

## **Table**:
- #### `dbo.Orders`

### **Description**:<br><p>A query that computes for each customer order both the difference between the current order quantity and the customer's previous order quantity AND the difference between the current order quantity and the customer's next order quantity:</p>

In [ ]:
USE	[TSQLV6]
GO

SELECT
	custid	AS	[Customer ID],
	orderid	AS	[Order ID],
	qty		AS	[Quantity],
	qty - LAG(qty)
		OVER(   PARTITION   BY	custid
                ORDER       BY	orderdate
        )   AS	[DiffFromPrevQty],
	qty - LEAD(qty)
		OVER(   PARTITION   BY	custid
                ORDER       BY	orderdate
        )   AS	[DiffFromNextQty]
FROM
	dbo.Orders;

# Exercise 4

## **Table**:
- #### `dbo.Orders`

### **Description**:<br><p>A query that returns a row for each employee, a column for each order year, and the count of orders for each employee and order year:</p>

In [ ]:
USE	[TSQLV6]
GO

SELECT 
	empid	AS	[Employee ID], 
	[2020], 
	[2021], 
	[2022]
FROM(
	SELECT
		empid,
		YEAR(orderdate) AS	[Order Year]
	FROM
		dbo.Orders
)	AS	d
PIVOT(
	COUNT([Order Year])
	FOR [Order Year]
        IN( [2020],
            [2021],
			[2022]
        )
)	AS	p
ORDER   BY
	empid;

# Exercise 5

### Run the following code to create and  populate the _EmpYearOrders_ table:
```SQL
DROP	TABLE	IF	EXISTS	dbo.EmpYearOrders;

CREATE	TABLE	dbo.EmpYearOrders(
	empid	INT     NOT	NULL
		CONSTRAINT	PK_EmpYearOrders	PRIMARY KEY,
	c2020	INT         NULL,
	c2021	INT         NULL,
	c2022	INT         NULL
);

INSERT	INTO	dbo.EmpYearOrders(
		empid,
		c2020,
		c2021,
		c2022
	)
SELECT	
	empid,
	[2020]	AS	c2020,
	[2021]	AS	c2021,
	[2022]	AS	c2022
FROM(
	SELECT
		empid,
		YEAR(orderdate)	AS	[Order Year]
	FROM
		dbo.Orders
)	AS	d
PIVOT(COUNT([Order Year])
	FOR	[Order Year]	IN( [2020],
							[2021],
							[2022])
)	AS	p;

SELECT	*	FROM	dbo.EmpYearOrders;
```

## **Tables**:
- #### `dbo.Orders`
- #### `dbo.EmpYearOrders`

### **Description**:<br><p>A query that unpivots the data, returning a row for each employee and order year with the number of orders.<br>Exclude rows in which the number of orders is 0:</p>

In [ ]:
USE [TSQLV6]
GO

SELECT
    empid		AS	[Employee ID],
    orderyear	AS	[Order Year],
    numorders	AS	[Number of Orders]
FROM
    EmpYearOrders
CROSS	APPLY	(
    VALUES		('2020',	c2020),
				('2021',	c2021),
				('2022',	c2022)
)	AS	unpvt(orderyear,	numorders)
WHERE
    numorders	IS	NOT	NULL
		AND
	numorders	<>	0;

# Exercise 6

## **Table**:
- #### `dbo.Orders`

### **Description**:<br><p>A query that returns the total quantities for each of the following:
- (employee, customer, and order year)
- (employee and order year)
- (customer and order year)
### Include a result column in the output that uniquely identifies the grouping sets with which the current row is associated:</p>

In [ ]:
USE	[TSQLV6]
GO

SELECT
	GROUPING_ID(
		empid,
		custid,
		YEAR(orderdate)
	)				AS	[Grouping Set],
	empid			AS	[Employee ID],
	custid			AS	[Customer ID],
	YEAR(orderdate)	AS	[Order Year],
	SUM(qty)		AS	[Total Quantity]
FROM
	dbo.Orders
GROUP   BY  GROUPING
	SETS(
		(empid, custid, YEAR(orderdate)),
		(empid,         YEAR(orderdate)),
		(       custid, YEAR(orderdate))
	);

# Exercise 7

## **Table**:
- #### `Sales.Orders`

### **Description**:<br><p>A query that returns a row for each week, assuming the week starts on a Sunday, with result columns showing when the week started, when the week ended, and the week's order count:</p>

In [ ]:
USE	[TSQLV6]
GO

WITH WeeklyOrders   AS  (
    SELECT 
        DATEADD(DAY,
				-DATEPART   (WEEKDAY,   orderdate)   +   1,
				orderdate
		)		AS	[Week Starting],
        DATEADD(DAY,
				7 - DATEPART(WEEKDAY,   orderdate),
				orderdate
		)		AS	[Week Ending],
        orderid AS	[Order ID]
    FROM 
        Sales.Orders
)
SELECT
    [Week Starting],
    [Week Ending],
    COUNT([Order ID])   AS  [Number of Orders]
FROM 
    WeeklyOrders
GROUP   BY
    [Week Starting],
    [Week Ending]
ORDER   BY
    [Week Starting];


# Exercise 8

## **Table**:
- #### `Sales.OrderValues`

### **Description**:<br><p>A query that returns the total quantities and values per shipper and fiscal year of the order date.<br>The result should have columns for the shipper ID, start date of fiscal year, end date of fiscal year, total quantity, and total value.<br>Assume the organization's fiscal year runs from July 1 to June 30.</p>

In [ ]:
USE	[TSQLV6]
GO

SELECT 
    shipperid		AS	[Shipper ID],
    [Start of Year],
    [End of Year],
    SUM(qty)		AS	[Total Quantity],
    SUM(val)		AS	[Total Value]
FROM(
    SELECT
        shipperid,
        IIF(MONTH(	orderdate)		>=	7,
			YEAR(	orderdate),
			YEAR(	orderdate)		-	1
		)					AS	[Fiscal Year],
        CAST(IIF(	MONTH(	orderdate)	>=	7,
					YEAR(	orderdate),
					YEAR(	orderdate)	-	1
			)	AS	VARCHAR(4)
		)	+	'-07-01'	AS	[Start of Year],
        CAST(IIF(	MONTH(orderdate)	>=	7,
					YEAR(orderdate)		+	1,
					YEAR(orderdate)
			)	AS	VARCHAR(4)
		)	+	'-06-30'	AS	[End of Year],
        qty,
        val
    FROM
		Sales.OrderValues
)	AS	fiscal_data
GROUP	BY
	shipperid,
	[Fiscal Year],
	[Start of Year],
	[End of Year]
ORDER	BY
	shipperid,
	[Fiscal Year];